In [1]:
#Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV , ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
import calendar
from pandas.api.types import CategoricalDtype
from plotnine import *
import plotly as py
from plotly.offline import iplot, plot, init_notebook_mode, download_plotlyjs
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.offline as offline
#import ggplot
from plotnine.data import mpg
# import import_ipynb
import nbimporter
# import fn as fn
%matplotlib inline


  question 1: Best time to visit Seattle and Boston  
  question 2: Neighbourhood analysis  
 question 3: Features affecting price 

In [2]:
# Loading Seattle Data
sea_list=pd.read_csv("sea_listings.csv")
sea_cal=pd.read_csv("sea_calendar.csv")


In [3]:
#Loading Boston Dataset
bos_list=pd.read_csv("bos_listings.csv")
bos_cal=pd.read_csv("bos_calendar.csv")

## Preprocessing Seattle Data 

In [4]:
sea_list.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48


In [5]:
sea_cal.head(2)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00


In [6]:
# converting id to listing_id so to merge both the listing and calender dataset of seattle
sea_list=sea_list.rename(columns={'id':'listing_id'})
sea_list_cal=pd.merge(sea_list, sea_cal, on='listing_id', how='inner')

In [7]:
sea_list_cal.head(2)

,listing_id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,date,available,price_y
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,WASHINGTON,f,moderate,f,f,2,4.07,2016-01-04,t,$85.00
1,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,WASHINGTON,f,moderate,f,f,2,4.07,2016-01-05,t,$85.00


In [8]:
sea_list_cal["available"] = sea_list_cal["available"].map(lambda x: 1 if x=="t" else 0)
sea_list_cal["price_y"]=sea_list_cal.price_y.str.replace('$','').str.replace(',','').astype(float)
sea_list_cal["weekly_price"]=sea_list_cal.weekly_price.str.replace('$','').str.replace(',','').astype(float)
sea_list_cal["monthly_price"]=sea_list_cal.monthly_price.str.replace('$','').str.replace(',','').astype(float)
sea_list_cal["security_deposit"]=sea_list_cal.security_deposit.str.replace('$','').str.replace(',','').astype(float)
sea_list_cal["cleaning_fee"]=sea_list_cal.cleaning_fee.str.replace('$','').str.replace(',','').astype(float)
sea_list_cal["host_response_rate"]=sea_list_cal.host_response_rate.str.replace('%','').str.replace(',','').astype(float)
sea_list_cal["host_acceptance_rate"]=sea_list_cal.host_acceptance_rate.str.replace('%','').str.replace(',','').astype(float)
sea_list_cal["extra_people"]=sea_list_cal.extra_people.str.replace('$','').str.replace(',','').astype(float)

In [9]:
pd.set_option('display.max_rows', 100) #https://stackoverflow.com/questions/23168416/pandas-printing-all-dtypes

In [10]:
# 
sea_list_cal['date'] = pd.to_datetime(sea_list_cal['date'])
sea_list_cal['host_since'] = pd.to_datetime(sea_list_cal['host_since'])
sea_list_cal['month']=sea_list_cal.date.dt.month.apply(lambda x: calendar.month_abbr[x])
sea_list_cal['year']=sea_list_cal.host_since.dt.year


## Preprocessing Boston Data

In [36]:
bos_list.head(1)

,listing_id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN


In [12]:
bos_cal.head(2)

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN


In [13]:
# converting id to listing_id so to merge both the listing and calender dataset of boston
bos_list=bos_list.rename(columns={'id':'listing_id'})
bos_list_cal=pd.merge(bos_list, bos_cal, on='listing_id', how='inner')

In [14]:
bos_list_cal["price_y"]=bos_list_cal.price_y.str.replace('$','').str.replace(',','').astype(float)
bos_list_cal["weekly_price"]=bos_list_cal.weekly_price.str.replace('$','').str.replace(',','').astype(float)
bos_list_cal["monthly_price"]=bos_list_cal.monthly_price.str.replace('$','').str.replace(',','').astype(float)
bos_list_cal["security_deposit"]=bos_list_cal.security_deposit.str.replace('$','').str.replace(',','').astype(float)
bos_list_cal["cleaning_fee"]=bos_list_cal.cleaning_fee.str.replace('$','').str.replace(',','').astype(float)
bos_list_cal["host_response_rate"]=bos_list_cal.host_response_rate.str.replace('%','').str.replace(',','').astype(float)
bos_list_cal["host_acceptance_rate"]=bos_list_cal.host_acceptance_rate.str.replace('%','').str.replace(',','').astype(float)
bos_list_cal["extra_people"]=bos_list_cal.extra_people.str.replace('$','').str.replace(',','').astype(float)

In [15]:
bos_list_cal["available"] = bos_list_cal["available"].map(lambda x: 1 if x=="t" else 0)
bos_list_cal['date'] = pd.to_datetime(bos_list_cal['date'])
bos_list_cal['host_since'] = pd.to_datetime(bos_list_cal['host_since'])
bos_list_cal['month']=bos_list_cal.date.dt.month.apply(lambda x: calendar.month_abbr[x])
bos_list_cal['year']=bos_list_cal.host_since.dt.year

In [16]:
sea_list_cal["available"] = sea_list_cal["available"].map(lambda x: 1 if x=="t" else 0)
bos_list_cal["available"] = bos_list_cal["available"].map(lambda x: 1 if x=="t" else 0)

### Q1. What is the best time to visit Seattle and Boston?


In [17]:
#Calculate monthly price by grouping by month and aggregating price_y in both seattle and boston dataframes

sea_monthly_price = sea_list_cal.groupby(['month'])['price_y'].mean().to_frame(name='seattle_price').reset_index()
bos_monthly_price = bos_list_cal.groupby(['month'])['price_y'].mean().to_frame(name='boston_price').reset_index()

In [18]:
# Creatig a list of months

months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Map the average calculation for each listed month 

m = {month: i for i, month in enumerate(months)}
key_s = sea_monthly_price['month'].map(m)    
key_b = bos_monthly_price['month'].map(m)    
sea_monthly_price = sea_monthly_price.iloc[key_s.argsort()]
bos_monthly_price = bos_monthly_price.iloc[key_b.argsort()]

In [19]:
trace1 = go.Scatter(
    x = sea_monthly_price.month[:-1],
    y = sea_monthly_price.seattle_price[:-1]
)
layout1 = go.Layout(
    title = "Average Prices by Month in Seattle",
    xaxis = dict(title = 'Month'),
    yaxis = dict(title = 'Price ($)')
)
data1 = [trace1]
figure1 = go.Figure(data = data1, layout = layout1)
offline.iplot(figure1)

In [20]:
trace2 = go.Scatter(
    x = bos_monthly_price.month,
    y = bos_monthly_price.boston_price
)
layout2 = go.Layout(
    title = "Average Prices by Month in Boston",
    xaxis = dict(title = 'Month'),
    yaxis = dict(title = 'Price ($)')
)
data2 = [trace2]
figure2 = go.Figure(data = data2, layout = layout2)
offline.iplot(figure2)

Observation:
From the first graph(Seattle) we observe that min average price of listing is January, then there is increase with maximum average around July. Fom Google search
From the secon graph(Boston) we observe that the avg listing price remain more or less same till March, then it started increasing with maximum average in September.

## Properties

In [21]:
sea_property_price = sea_list_cal.groupby(['property_type'])['price_y'].mean().to_frame(name='sea_price').reset_index().sort_values(by = "sea_price", axis=0, ascending=False) 
bos_property_price = bos_list_cal.groupby(['property_type'])['price_y'].mean().to_frame(name='bos_price').reset_index().sort_values(by = "bos_price", axis=0, ascending=False)

In [22]:
trace3 = go.Bar(
    x = sea_property_price.property_type[:-1],
    y = sea_property_price.sea_price[:-1]
)
layout3 = go.Layout(
    title = "Average Prices of Properties in Seattle",
    xaxis = dict(title = 'property_type	'),
    yaxis = dict(title = 'Price ($)')
)
data3 = [trace3]
figure3 = go.Figure(data = data3, layout = layout3)
offline.iplot(figure3)

In [23]:
trace4 = go.Bar(
    x = bos_property_price.property_type[:-1],
    y = bos_property_price.bos_price[:-1]
)
layout4 = go.Layout(
    title = "Average Prices of Properties in Boston",
    xaxis = dict(title = 'property_type	'),
    yaxis = dict(title = 'Price ($)')
)
data4= [trace4]
figure4 = go.Figure(data = data4, layout = layout4)
offline.iplot(figure4)

Observation

## Neighbourhoods

In [24]:
sea_neighbourhood_price = sea_list_cal.groupby(['neighbourhood'])['price_y'].mean().to_frame(name='sea_price').reset_index().sort_values(by = "sea_price", axis=0, ascending=False) 
bos_neighbourhood_price = bos_list_cal.groupby(['neighbourhood'])['price_y'].mean().to_frame(name='bos_price').reset_index().sort_values(by = "bos_price", axis=0, ascending=False)

In [25]:
top_10_sea_neighbourhood = sea_neighbourhood_price[:10]

In [26]:
trace5 = go.Bar(
    x = top_10_sea_neighbourhood.neighbourhood,
    y = top_10_sea_neighbourhood.sea_price
)
layout5 = go.Layout(
    title = "Average Prices of Neighbourhoods in Seattle",
    xaxis = dict(title = 'Neighbourhood'),
    yaxis = dict(title = 'Price ($)')
)
data5= [trace5]
figure5 = go.Figure(data = data5, layout = layout5)
offline.iplot(figure5)

In [27]:
top_10_bos_neighbourhood = bos_neighbourhood_price[:10]

In [28]:
trace6 = go.Bar(
    x = top_10_bos_neighbourhood.neighbourhood[:-1],
    y = top_10_bos_neighbourhood.bos_price[:-1]
)
layout6 = go.Layout(
    title = "Average Prices of Neighbourhoods in Boston",
    xaxis = dict(title = 'Neighbourhood	'),
    yaxis = dict(title = 'Price ($)')
)
data6= [trace6]
figure6 = go.Figure(data = data6, layout = layout6)
offline.iplot(figure6)

## Room Types

In [29]:
sea_rooms_price = sea_list_cal.groupby(['room_type'])['price_y'].mean().to_frame(name='sea_price').reset_index() 
bos_rooms_price = bos_list_cal.groupby(['room_type'])['price_y'].mean().to_frame(name='bos_price').reset_index()

In [30]:
trace7 = go.Bar(
    x = sea_rooms_price.room_type,
    y = sea_rooms_price.sea_price
)
layout7 = go.Layout(
    title = "Room type vs Price in Seattle",
    xaxis = dict(title = 'Room types	'),
    yaxis = dict(title = 'Price ($)')
)
data7= [trace7]
figure7 = go.Figure(data = data7, layout = layout7)
offline.iplot(figure7)

In [31]:
trace8 = go.Bar(
    x = bos_rooms_price.room_type,
    y = bos_rooms_price.bos_price
)
layout8 = go.Layout(
    title = "Room type vs Price in Boston",
    xaxis = dict(title = 'Room types	'),
    yaxis = dict(title = 'Price ($)')
)
data8= [trace8]
figure8 = go.Figure(data = data8, layout = layout8)
offline.iplot(figure8)

## Number of Bedrooms

In [33]:
sea_bedrooms_price = sea_list_cal.groupby(['bedrooms'])['price_y'].mean().to_frame(name='sea_price').reset_index().sort_values(by = "sea_price", axis=0, ascending=False) 
bos_bedrooms_price = bos_list_cal.groupby(['bedrooms'])['price_y'].mean().to_frame(name='bos_price').reset_index().sort_values(by = "bos_price", axis=0, ascending=False)

In [34]:
trace9 = go.Bar(
    x = sea_bedrooms_price.bedrooms[:-1],
    y = sea_bedrooms_price.sea_price[:-1]
)
layout9 = go.Layout(
    title = "No. of bedrooms vs Price in Seattle",
    xaxis = dict(title = 'No. of bedrooms	'),
    yaxis = dict(title = 'Price ($)')
)
data9= [trace9]
figure9 = go.Figure(data = data9, layout = layout9)
offline.iplot(figure9)

In [35]:
trace10 = go.Bar(
    x = bos_bedrooms_price.bedrooms[:-1],
    y = bos_bedrooms_price.bos_price[:-1]
)
layout10 = go.Layout(
    title = "No. of bedrooms vs Price in Boston",
    xaxis = dict(title = 'No. of Bedrooms'),
    yaxis = dict(title = 'Price ($)')
)
data10= [trace10]
figure10 = go.Figure(data = data10, layout = layout10)
offline.iplot(figure10)